In [ ]:
%ls data/dogscats_new
%matplotlib inline

In [ ]:
data_path = "data/dogscats_new/"

In [ ]:
import numpy as np
import math
import cv2
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from keras.optimizers import  Adam
from keras.models import Sequential, Model
from keras.layers import Dense


# Get data for training and validation
Import and init ImageData generator for future image processing from directory

In [ ]:

train_imgs = ImageDataGenerator()
valid_imgs = ImageDataGenerator()
test_imgs = ImageDataGenerator()

# Load VGG16 keras model

In [ ]:
vgg16 = VGG16()

### Remove last layer from vgg16
Since this model is based on imagenet but we want to have only 2 categories instead for 1000 like imagenet have

In [ ]:

vgg16.layers.pop()
for layer in vgg16.layers: 
    layer.trainable=False

last = vgg16.layers[-1].output
x = Dense(2, activation="softmax")(last)
model = Model(vgg16.input, x)

model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'] )

In [ ]:
img_size = 224 # because vgg16 input is 224,244,3 ; 3 here is for RGB 
batch_size = 64
img_batch_size = 8
train_imgs_gen = train_imgs.flow_from_directory(data_path + "train", 
                                                target_size=(img_size,img_size), 
                                                batch_size=img_batch_size,
                                                color_mode = "rgb",
                                                class_mode='categorical',
                                                shuffle=True)

valid_imgs_gen = valid_imgs.flow_from_directory(data_path + "valid", 
                                                target_size=(img_size,img_size), 
                                                batch_size=img_batch_size,
                                                color_mode = "rgb",
                                                class_mode='categorical',
                                                shuffle=True)

test_imgs_gen = test_imgs.flow_from_directory(data_path + "test",
                                              target_size=(img_size,img_size), 
                                              batch_size=img_batch_size,
                                              color_mode = "rgb",
                                              class_mode=None,
                                              shuffle=False)
#model.load_weights('my_custom_vgg16_model_sgd_4.weight')

In [ ]:
for count in range(5):
    model.fit_generator(  train_imgs_gen, 
                          samples_per_epoch= train_imgs_gen.nb_sample,
                          nb_epoch = 1,
                          validation_data = valid_imgs_gen,
                          nb_val_samples = valid_imgs_gen.nb_sample
                         )
    model.save_weights(f'my_custom_vgg16_model_adam_{count}.weight')

In [ ]:
from PIL import Image

img = cv2.imread('./data/dogscats_new/test/unknown/333.jpg')
img=cv2.resize(img,(224,224))
img=np.array(img).reshape((224,224,3))
plt.imshow(img)
img = img.reshape((1,) + img.shape)
np.rint(model.predict(img))


In [ ]:
model_gen = model.predict_generator(test_imgs_gen, test_imgs_gen.nb_sample)

In [ ]:
submission_file_name = 'submission_vgg16_5epoch.csv'
filenames = test_imgs_gen.filenames
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
isdog = model_gen[:,1]
isdog = isdog.clip(min=0.05, max=0.95)
subm = np.stack([ids,isdog], axis=1)
subm[:5]
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')

In [ ]:
from IPython.display import FileLink
FileLink(submission_file_name)